In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
import json
import html5lib
import time
import glob

In [2]:
#setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [C:\Users\delac\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


# Retrieve the CSV's and Convert to DataFrames

In [3]:
# Setup the URL to visit
url = 'https://aflsbkg.airfrance.fr/'

# Visit the URL
browser.visit(url)

In [4]:
# Store origin and Destination for Airfrance and KLM
AF_flight = ["ORD", "CDG"]
KL_flight =  ["ORD", "AMS"]

# Store the date Variable (day, month, year)
day = list(map(int, input("What day(s) do you wish to screen").split(",")))
month = input("Input the month in 3 word format (Ex = Apr)").title()
year = "2021"

What day(s) do you wish to screen21
Input the month in 3 word format (Ex = Apr)Jun


In [ ]:
# navigate to the correct webpage for CSV download
browser.links.find_by_partial_text('Booking').click()
browser.links.find_by_partial_text('Search').click()

time.sleep(2.00)

# fill the flight number and date input field
browser.fill("segment.segOrigin", f"{AF_flight[0]}")
browser.fill("segment.segDestination", f"{AF_flight[1]}")
browser.fill("bookingSearch.legDepartureDate", f"{day[0]}/{month}/{year}")

# filter the search for the flight needed
browser.links.find_by_partial_text('Filter').click()

time.sleep(3.00)

# browser.click_link_by_id('FILTER_EXPAND_COLLAPSE')
#Visit inital page and scrape the first page data
html = browser.html
soup = bs(html, 'html.parser')
table = soup.find_all('table', class_= 'listTable')
df = pd.read_html(str(table))[0]

# Create a for loop to visit the other pages of data and concat with original df
num = 2
for y in range(10):
    try:
        browser.click_link_by_partial_href(f'newbookListPagingProcess.do?page={num}')
        html1 = browser.html
        soup1 = bs(html1, 'html.parser')
        table1 = soup1.find_all('table', class_= 'listTable')
        df2 = pd.read_html(str(table1))[0]
        df = pd.concat([df, df2], ignore_index=True)
        num += 1
    except:
        break
AF = df[(df.Prefix == '006') | (df.Prefix == '057')].drop(['Unnamed: 0'], axis = 1)

browser.links.find_by_partial_text('Booking').click()
browser.links.find_by_partial_text('Search').click()

time.sleep(3.00)

browser.fill("segment.segOrigin", f"{KL_flight[0]}")
browser.fill("segment.segDestination", f"{KL_flight[1]}")
browser.fill("bookingSearch.legDepartureDate", f"{day[0]}/{month}/{year}")


# filter the search for the flight needed
browser.links.find_by_partial_text('Filter').click()

time.sleep(3.00)

# browser.click_link_by_id('FILTER_EXPAND_COLLAPSE')
#Visit inital page and scrape the first page data
html = browser.html
soup = bs(html, 'html.parser')
table = soup.find_all('table', class_= 'listTable')
df = pd.read_html(str(table))[0]

# Create a for loop to visit the other pages of data and concat with original df
num = 2
for y in range(10):
    try:
        browser.click_link_by_partial_href(f'newbookListPagingProcess.do?page={num}')
        html1 = browser.html
        soup1 = bs(html1, 'html.parser')
        table1 = soup1.find_all('table', class_= 'listTable')
        df2 = pd.read_html(str(table1))[0]
        df = pd.concat([df, df2], ignore_index=True)
        num += 1
    except:
        break
KL = df[(df.Prefix == '057') |(df.Prefix == '074') ].drop(['Unnamed: 0'], axis = 1)

df_final = pd.concat([AF, KL], ignore_index = True)

C:\Users\delac\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:485: FutureWarning: browser.find_link_by_partial_href is deprecated. Use browser.links.find_by_partial_href instead.
  FutureWarning,


# Filter the Dataframe

In [ ]:
# Select the columns you want to display and store in a list

columns = ["Prefix", "AWB Number", "Origin","Destination","Commodity", \
           "Product","Pieces", "Weight","Volume","Issuer","Flight Info"]

# Filter the AWB's by  weight and volume 
df_final = df_final[(df_final.Weight.astype('float') > 1000) & (df_final.Volume.astype("float") > 10.00)][columns]

# Filter the AWB's by product
df_final = df_final[~df_final.Product.str.contains("S50|S51|S52|S53")].sort_values('Flight Info').reset_index(drop = True)

In [ ]:
df_test = pd.DataFrame(columns = columns)
# append an empty row for csv formating. 
df_test = df_test.append(pd.Series(name= " ")).fillna("")

# append the first flight that it shows 
df_test = df_test.append(pd.Series(name= df_final["Flight Info"][0])).fillna("")
past_flight = df_final["Flight Info"][0]

for index, row in df_final.iterrows():
    if past_flight != row["Flight Info"]:
        df_test = df_test.append(pd.Series(name= " ")).fillna("")
        df_test = df_test.append(pd.Series(name= row["Flight Info"])).fillna("")
        df_test = df_test.append(row)
    else:
        df_test = df_test.append(row)
    past_flight = row["Flight Info"]
df_test

In [ ]:
df_test.to_csv("test.csv")
len(df_final)

In [ ]:
df_final